# Prompt Optimizer

In this notebook we are building a prompt optimizer based upon the research whitepaper:  Principled Instructions Are All You Need for Questioning LLaMA-1/2, GPT-3.5/4  download from here https://arxiv.org/pdf/2312.16171.pdf


In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

In [ ]:
# import os module & the OpenAI Python library for calling the OpenAI API
import os
from openai import AzureOpenAI
import json
from dotenv import load_dotenv
load_dotenv()

# Setup Parameters

In [ ]:
# Expecting an .env file with the following content:
# OPENAI_API_KEY=your_openai_api_key_here
# AZURE_OPENAI_ENDPOINT ="https://your_open_ai_resource_name.openai.azure.com/"



In [ ]:

# System prompt for reviewing user prompts, adapted for the conversational format
reviwer_system_prompt = """
##Objective##:
To review user prompts critically, ensuring they align with the 26 key principles for effective interaction with Large Language Models (LLMs), and to provide constructive feedback for improvement.

##Instructions for the Assistant##
#Directness and Conciseness: Confirm the user prompt is straightforward, avoiding unnecessary politeness or filler words.
#Intended Audience: Ensure the user prompt clearly identifies who the audience is, tailoring the complexity and style accordingly.
#Simplifying Complex Tasks: Check if complex tasks are broken down into simpler, sequential prompts.
#Affirmative Directives: Assess the use of clear, action-oriented language, avoiding negative constructions.
#Clarity in Requests: Verify if the prompt asks for explanations in simple terms, suitable for laypersons or specific age groups.
#Incentive Mention: Note if the prompt includes unnecessary incentives, which are not applicable here.
#Example-Driven: Ensure the prompt encourages or includes specific examples or few-shot learning approaches.
#Structured Formatting: Check for clear, structured formatting, including instructional headers and separation of sections.
#Directive Phrasing: Confirm the use of explicit directives like “Your task is” to guide the LLM's response.
#Consequence Awareness: Look for language indicating awareness of the importance of accuracy and the potential consequences of misinformation.
#Natural, Human-Like Responses: Assess whether the prompt requests responses in a natural, conversational manner.
#Step-by-Step Guidance: Ensure the prompt encourages thinking or explaining processes step by step.
#Bias Avoidance: Check for explicit instructions to avoid bias and rely on stereotypes.
#Detail Elicitation: Confirm if the prompt is open-ended, allowing the LLM to ask follow-up questions for clarification.
#Learning and Testing: Verify if the prompt encourages an educational approach, including learning checks or tests.
#Role Assignment: Assess whether the prompt assigns a specific role or perspective to the LLM.
#Use of Delimiters: Check for the use of delimiters for clarity in separating sections or ideas.
#Keyword Repetition: Look for intentional repetition of key words or phrases for emphasis.
#Combining CoT with Few-Shot: Ensure the prompt integrates a Chain-of-Thought approach with example-driven or few-shot learning for complex tasks.
#Output Primers: Confirm if the prompt concludes with an output primer, setting up the expected format or beginning of the response.
#Detailed Writing Requests: Assess if the prompt asks for detailed, comprehensive coverage on a topic.
#Style Preservation: Ensure the prompt asks for stylistic consistency when revising or responding to user inputs.
#Complex Coding Requests: Note if the prompt properly guides the creation of multi-file coding solutions or complex coding tasks.
#Content Initiation: Verify if the prompt effectively sets the stage or beginning for creative or continuation tasks.
#Clear Requirements: Ensure the prompt clearly outlines all requirements, including format, style, and specific content expectations.
#Style Mimicry: Check if the prompt asks for mimicry of a provided sample's style or language for text generation tasks.

##Feedback and Recommendations##

#Provide specific, actionable feedback for each principle, noting where the user prompt excels and where it falls short.
#Offer suggestions for improvement, ensuring the user understands how to align their prompt better with these principles.
#YOU MUST ALWAYS GENERATE AN IMPROVED VERSION OF THE PROMPT AND SUGGEST IT TO THE USER
##Final Evaluation##

Give an overall evaluation of the prompt's effectiveness and adherence to the 26 principles.
Encourage the user to refine their prompting skills based on the feedback.

"""


prompt_to_review = """
Develop an executive summary that encapsulates the potential investment in a point-of-care testing company within the in-vitro diagnostics market. The summary should:

Present an overview of the company's business model, product offerings, market reach, and financial highlights without specifying exact sales figures or margins.
Give an account of the company's historical background, current ownership, and ongoing bid process, mentioning interest from other equity firms without naming them.
Formulate an investment thesis that emphasizes the company's market position, growth strategy, and business model strengths, such as recurring revenue streams and proprietary technology.
List key investment considerations and due diligence items that an investor should evaluate, focusing on market share, product portfolio, and technological innovation.
Conclude with a preliminary valuation outlook and recommendations for proceeding with an investment procedure, without including specific multiples or revenue projections."
One-shot Example for LLM Output:

"The executive summary should highlight an investment opportunity in a dynamic IVD company with a strong presence in point-of-care testing. The company, leveraging a recurring revenue model, has established significant market access in Europe and plans for global expansion. Originally part of a larger pharmaceutical entity, it is now in the midst of a strategic acquisition process. The investment thesis should underscore the firm's solid market standing, the potential for growth via technological advancements, and its strategic plans for market expansion. Considerations for investment should cover market analysis, risk assessment, and evaluation of proprietary technologies. The summary concludes with a recommendation for investment proceedings based on a positive valuation outlook, considering the company's robust business model and future potential."
"""
messages =[
    {
        "role": "system",
        "content": reviwer_system_prompt
    },
    {
        "role": "user",
        "content": prompt_to_review
    }
]
   



In [ ]:

try:
    
    client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

    deployment_name='gpt-4' #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 
        
    # Send a completion call to generate an answer
    completion = client.chat.completions.create(
        messages=messages,
        model=deployment_name,
        max_tokens=3000
    )

    print(f"{completion.choices[0].message.role}: {completion.choices[0].message.content}")
except Exception as e:
    # Handles all other exceptions
    print(f"An exception has occurred: {e}")
